In [ ]:
##### EL CÓDIGO CONSISTE DE DOS PARTES:
##### 1. OBTENER EL MOVIMIENTO DIARIO DE LAS ACCIONES 
##### BAJO EL TÍTULO: "WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL"
##### 2. OBTENER LA INFORMACIÓN FINANCIERA DE LAS EMPRESAS LISTADAS EN LA BVL (LAS QUE ESCOJA)
##### BAJO EL TÍTULO: "WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE UNA EMPRESA PERUANA DE LA BVL"
########## NOTA 1: DEBE TOMAR EN CUENTA QUE LAS PÁGINAS ESTÁN SUJETAS A CAMBIOS,
########## POR LO QUE EN EL FUTURO EL CÓDIGO PODRÍA REQUERIR CIERTAS VARIACIONES
########## NOTA 2: ALGUNOS PROCESOS TOMAN CIERTO TIEMPO PARA GARANTIZAR UNA CORRECTA EXTRACCIÓN DE DATOS.
########## NOTA 3: ESTE ES UN CÓDIGO LIBRE ( YA SEA COMO MATERIAL DE APRENDIZAJE O SI DESEA REALIZAR SU PROPIA VERSIÓN)
########## NOTA 4: SIEMPRE HAY ESPACIO PARA MEJORAS Y ACTUALIZACIONES
########## NOTA 5: ASEGÚRESE DE SEGUIR LAS INTRUCCIONES Y TOMAR EN CUENTA LOS COMENTARIOS

In [1]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########

#No es necesario ejecutar esto, puede hacerlo si desea ver un poco de lo que se utilizará más adelante
#Importando las librerías necesarias
#En caso no se tengan las librerías, instalarlas
#pip install BeautifulSoup
#pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from csv import writer
import time
import openpyxl
import pandas as pd

# Inicializar el controlador de Selenium
#Si tienes alguna duda sobre la instalación de chromedriver y de la ruta que debes utilizar, puedes revisar el siguiente link:
# https://www.youtube.com/watch?v=57ukgFoHSy0

driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe") # Reemplazar con la ruta al controlador de Chrome

# Abrir la página web
url = "https://www.bvl.com.pe/mercado/movimientos-diarios"
driver.get(url)
driver.maximize_window()

# tiempo de espera de 5 segundos antes de ejecutar la siguiente línea de código
time.sleep(5)
# Cerrar el controlador de Selenium
driver.quit() 

C:\Users\Administrador\AppData\Local\Temp\ipykernel_31052\3104424959.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe") # Reemplazar con la ruta al controlador de Chrome


In [ ]:
####### WEB SCRAPING DE LOS MOVIMIENTOS DIARIOS DE LA BVL ########
####### Guardando los datos en un Excel XLSX

import openpyxl
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe")
driver.get(url)
driver.maximize_window()
time.sleep(3)
    
driver.execute_script("window.scrollBy(0, 300);")
time.sleep(5)    

#Cerrando el botón de los cookies, si no le aparece solo comente esta línea de código o bórrelo
actions = ActionChains(driver)
boton_cookie1=driver.find_element(By.XPATH, '/html/body/bvl-root/bvl-main-layout/bvl-footer/footer/bvl-cookies-banner/div/div/div/div[2]/button')
boton_cookie1.click()
time.sleep(2)  

# Crear un archivo de Excel nuevo
workbook = openpyxl.Workbook()

# Seleccionar la primera hoja
sheet = workbook.active
    
# Agregando los encabezados    
encabezado = driver.find_elements(By.XPATH, '//*[@id="page-view"]/main/section[1]/bvl-daily-table/div/section/div/div/article/div/div/table/thead/tr/th')
header = [elem.text for elem in encabezado]
sheet.append(header)
 
#Número de páginas en el registro del día
botones= driver.find_elements(By.XPATH, '//*[@id="page-view"]/main/section[1]/bvl-daily-table/div/section/footer/bvl-pagination/div/div/div/div/div[2]/div/button')
numero_botones= len(botones)


#Número de filas y columnas totales
rows = len(driver.find_elements(By.XPATH, '//*[@id="page-view"]/main/section[1]/bvl-daily-table/div/section/div/div/article/div/div/table/tbody/tr/td[1]'))
col = len(driver.find_elements(By.XPATH, '//*[@id="page-view"]/main/section[1]/bvl-daily-table/div/section/div/div/article/div/div/table/tbody/tr[1]/td'))-1
    
#Creamos una lista vacía
fila =[]

#Iterams a través de cada página para conseguir la información
for elementos in botones:
    
    if rows <= 50:
        fil = rows
    elif int(elementos.text) < numero_botones:
        fil=49 #número máximo de elementos encontrados por página al momento
    else:
        fil=rows-(numero_botones - 1)*50
        
        
    time.sleep(3)
    fila=[]
    
    if elementos.text==1:
        time.sleep(3)
        for n in range(1, fil+1):
            if n % 2 != 0:
                fila = []
                for b in range(1, col+1):
                    #time.sleep(2)
                    dato = driver.find_element(By.XPATH, "//*[@id='page-view']/main/section[1]/bvl-daily-table/div/section/div/div/article/div/div/table/tbody/tr["+str(n)+"]/td["+str(b)+"]")
                    #dato=dato.text.replace("S/", "").replace("US$", "") #retira el # si desea retirar los símbolos en la data
                    fila.append(dato.text) # si retira el comentario reemplace por: fila.append(dato)          
                sheet.append(fila) 
            else:
                continue        
    else:
        while not elementos.is_displayed():
            driver.execute_script("window.scrollBy(0, 300);")
            time.sleep(3)       
        driver.execute_script("arguments[0].click();", elementos)
        time.sleep(3)                      
        for m in range(1, fil+1):
            if m % 2 != 0:
                fila = []
                for n in range(1, col+1):
                    dato = driver.find_element(By.XPATH, "//*[@id='page-view']/main/section[1]/bvl-daily-table/div/section/div/div/article/div/div/table/tbody/tr["+str(m)+"]/td["+str(n)+"]")
                    #dato=dato.text.replace("S/", "").replace("US$", "") #retira el # si desea retirar los símbolos en la data
                    fila.append(dato.text) # si retira los comentarios, comente esta línea con # 
                    #fila.append(dato)
                sheet.append(fila) 
            else:
                continue

                
#Cerramos la ventana
driver.quit()

# Guardar el archivo de Excel
workbook.save('Daily_Movements_BVL.xlsx')
#Para cuestiones adicionales, sugiero revisar la Nota en el archivo excel subido en el proyecto.

#Ahora podemos convertirlo en un dataframe o CSV según la preferencia
# Leer el archivo Excel
df = pd.read_excel("Daily_Movements_BVL.xlsx")

# Convertir el DataFrame a CSV
df.to_csv("Daily_Movements_BVL.csv", index=False)

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAS PERUANA DE LA BVL ########
#Se obtienen los Estados Financieros de las empresas listadas en la BVL y que se elija

#Primero obtendremos los enlaces que contienen la información financiera

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from csv import writer
import time
import openpyxl
import pandas as pd

driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe")
driver.get(url)
driver.maximize_window()
time.sleep(3)
    
driver.execute_script("window.scrollBy(0, 300);")
time.sleep(3)    

#Cerrando el botón de los cookies, si no le aparece solo comente el código o bórrelo
actions = ActionChains(driver)
boton_cookie1=driver.find_element(By.XPATH, '/html/body/bvl-root/bvl-main-layout/bvl-footer/footer/bvl-cookies-banner/div/div/div/div[2]/button')
boton_cookie1.click()
time.sleep(3) 

paginas = driver.find_elements(By.XPATH, '//*[@id="page-view"]/main/section[1]/bvl-daily-table/div/section/div/div/article/div/div/table/tbody/tr/td/article/div/div[2]/bvl-links-list/ul/li[5]/a')


# crear lista para almacenar URLs
informacion = []

# recorrer elementos y obtener URLs
for pagina in paginas:
    url = pagina.get_attribute('href')
    informacion.append(url)

#Podemos imprimir los links y ver cuántos obtuvimos
#print(informacion)    
#print(len(informacion))

#Nos aseguramos de que no haya repetidos
informacion_financiera = list(set(informacion))
print(informacion_financiera)
print()
print(f"Se tienen {len(informacion_financiera)} enlaces")

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAS PERUANAS DE LA BVL ########

#Enlazamos los links a la empresa correspondiente (nombre de la empresa y símbolo de la acción)
empresas = {}  # Diccionario vacío para guardar la información

for url in informacion_financiera:
    # Inicializar el controlador de Selenium
    driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe") # Reemplazar con la ruta al controlador de Chrome

    # Abrir la página web
    driver.get(url)
    driver.maximize_window()
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, -850);")
    
    #Cierras los cookies
    actions = ActionChains(driver)
    boton_cookie1=driver.find_element(By.XPATH, '/html/body/bvl-root/bvl-main-layout/bvl-footer/footer/bvl-cookies-banner/div/div/div/div[2]/button')
    boton_cookie1.click()

    # Esperar a que cargue la página, agregar más tiempo en caso no le ejecuta
    time.sleep(4)

    # Obtener el nombre de la empresa
    company_name = driver.find_element(By.XPATH, '//*[@id="company-name"]').text
    #print(company_name)

    # Obtener el acrónimo de la acción
    symbol = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/bvl-details-chart/div/section/header/div/div/div[1]/div[1]/div/div[1]/span').text
    #print(symbol)

    # Agregar la información al diccionario
    empresas[url] = {'nombre': company_name, 'simbolo': symbol}

    driver.quit() #Cerrar el navegador
    #time.sleep(2)
    
print(empresas)

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAS PERUANAS DE LA BVL ########

#Elija las empresas de la que desea buscar su Información Financiera
#Se obtienen los nombres, símbolo de la acción y los links
#Asegúrese de seguir las instrucciones
#Las listas (ya sea de nombres o símbolos) puedes utilizarlo para realizar la búsqueda
#Toma en cuenta que es una búsqueda exacta (referencia al anterior comentario)

#Creando la función de búsqueda
def buscar_empresas():
    empresas_seleccionadas = {}
    contador=1
    while True:
        opcion_busqueda = input("¿Desea buscar por nombre o por símbolo? Escriba N para buscar por nombre o S para buscar por símbolo: ")
        if opcion_busqueda.upper() == 'N':
            opcion_lista = input("¿Desea ver la lista de nombres de empresas? Escriba S para sí o N para no: ")
            if opcion_lista.upper() == 'S':
                print("Lista de empresas:")
                for empresa in sorted(empresas.values(), key=lambda x: x['nombre']):
                    print("- " + empresa['nombre'])
            nombre_empresa = input("Escriba el nombre de la empresa que desea buscar: ")
            empresas_encontradas = [url_empresa for url_empresa, datos_empresa in empresas.items() if datos_empresa['nombre'] == nombre_empresa.upper()]
            while len(empresas_encontradas) > 1:
                print("Se encontraron varias empresas con el mismo nombre, por favor indique el símbolo de la empresa que desea buscar:")
                opcion_lista_simbolos = input("¿Desea ver la lista de símbolos de empresas? Escriba S para sí o N para no: ")
                if opcion_lista_simbolos.upper() == 'S':
                    print("Lista de símbolos de empresas:")
                    for empresa in sorted(empresas.values(), key=lambda x: x['nombre']):
                        print("- " + empresa['nombre'])
                simbolo_empresa = input("Escriba el símbolo de la empresa que desea buscar: ")
                empresas_encontradas = [url_empresa for url_empresa, datos_empresa in empresas.items() if datos_empresa['nombre'] == nombre_empresa and datos_empresa['simbolo'] == simbolo_empresa.upper()]
            if len(empresas_encontradas) == 0:
                print("No se encontró la empresa.")
            else:
                print("Puedes encontrar información financiera de la empresa en el siguiente link:", empresas_encontradas[0])
                empresa_encontrada = empresas_encontradas[0]
                datos_empresa = empresas[empresa_encontrada]
                numero_empresa= empresas_encontradas[0]
                numero_empresa = 'empresa' + str(contador)
                datos_empresa_seleccionada = {'nombre': datos_empresa['nombre'], 'simbolo': datos_empresa['simbolo'], 'link':empresas_encontradas[0]}
                empresas_seleccionadas[numero_empresa] = datos_empresa_seleccionada
                contador += 1
        elif opcion_busqueda.upper() == 'S':
            opcion_lista = input("¿Desea ver la lista de símbolos de empresas? Escriba S para sí o N para no: ")
            if opcion_lista.upper() == 'S':
                print("Lista de símbolos de empresas:")
                for empresa in empresas.values():
                    print("- " + empresa['simbolo'])
            simbolo_empresa = input("Escriba el símbolo de la empresa que desea buscar: ")
            empresas_encontradas = [url_empresa for url_empresa, datos_empresa in empresas.items() if datos_empresa['simbolo'] == simbolo_empresa.upper()]
            if len(empresas_encontradas) == 0:
                print("No se encontró la empresa.")
            else:
                print("Puedes encontrar información financiera de la empresa en el siguiente link:", empresas_encontradas[0])
                empresa_encontrada = empresas_encontradas[0]
                datos_empresa = empresas[empresa_encontrada]
                numero_empresa= empresas_encontradas[0]
                numero_empresa = 'empresa' + str(contador)
                datos_empresa_seleccionada = {'nombre': datos_empresa['nombre'], 'simbolo': datos_empresa['simbolo'], 'link':empresas_encontradas[0]}
                empresas_seleccionadas[numero_empresa] = datos_empresa_seleccionada
                contador += 1

        else:
            print("Opción no válida.")

        opcion_continuar = input("¿Desea continuar con la búsqueda? Escriba S para sí o N para no: ")
        if opcion_continuar.upper() == 'N':
            print("\n")
            print("Estas son las empresas seleccionadas:")
            print(empresas_seleccionadas)
            return empresas_seleccionadas
            break
            
#Ejecutar la función de búsqueda y guardarla
empresas_seleccionadas= buscar_empresas()

In [ ]:
####### WEB SCRAPING DE LOS ESTADOS FINANCIEROS DE EMPRESAs PERUANAs DE LA BVL ########

from selenium import webdriver
from selenium.webdriver import ActionChains
from csv import writer
import time
import openpyxl
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import datetime

enlaces = [empresa['link'] for empresa in empresas_seleccionadas.values()]
nombres = [empresa['nombre'] for empresa in empresas_seleccionadas.values()] #VOLCAN COMPAÑIA MINERA S.A.A. #ALICORP S.A.A.

for i, url in enumerate(enlaces):
    driver = webdriver.Chrome(executable_path=r"C:\Users\Administrador\Desktop\chromedriver.exe")
    driver.get(url)
    driver.maximize_window()
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, 525);")
    
    Año_actual= datetime.datetime.now().year #Comienza desde 2023
    
    actions = ActionChains(driver)
    boton_cookie=driver.find_element(By.XPATH, '/html/body/bvl-root/bvl-main-layout/bvl-footer/footer/bvl-cookies-banner/div/div/div/div[2]/button')
    boton_cookie.click()
    time.sleep(2)
    
    #Seleccionar el Periodo
    time.sleep(3)
    periodo = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[2]/div/div[1]')
    time.sleep(2)
    anual = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[2]/div/div[2]/div[2]')
    time.sleep(3)
    actions = ActionChains(driver)
    actions.move_to_element(periodo).move_to_element(anual).click().perform()
    
    #Mover el cursor a filtro Año    
    año = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[3]/div/div[1]')
    time.sleep(2)
    
    libro = openpyxl.Workbook()
    
    #Se pregunta la cantidad de años que desea extraer, Ejm: 2018-2020 es 3
    while True:
        try:
            numero_años = int(input("Ingrese el número de años que desea obtener: "))
            break
        except ValueError:
            print("Número inválido. Por favor, ingrese un número entero válido.")   
            
    verificador=numero_años
    
    año_actual=1 #1 es el año más actual, 2 el año anterior y así sucesivamente
    
    while verificador>0:
            
        año = driver.find_element(By.XPATH, '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[3]/div/div[1]')
        time.sleep(2)
        
        #Seleccionamos el año
        año_elegido = driver.find_element(By.XPATH, "//*[@id='page-view']/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/bvl-toolbar/div/div[3]/div/div[2]/div["+str(año_actual)+"]")
        time.sleep(2)
        
        #Seleccionando filtro año
        if año_actual==1:
            actions.move_to_element(año).click()
            time.sleep(2) 
        else:
            actions.move_to_element(año).move_to_element(año_elegido).click().perform()
            time.sleep(3)
            
        año_actual = año_actual + 1         
        
        try:
            existe_informacion =driver.find_element(By.XPATH, "//*[@id='page-view']/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-section-header/header/div[1]/h1")             
        except NoSuchElementException:
            existe_informacion = None
        
        #print("La informacion es ", existe_informacion)   
                
        if existe_informacion is not None:
            
            #Contador que ayuda a verificar la cantidad de años extraídos
            verificador -= 1
            
            #Seleccionar Estado Financiero:
            #Elegimos los estados principales (si desea otra información, puede automatizar o cambiar valores)
            Estado_Situacion_Financiera = '//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-table-file-link/table/tbody/tr[1]/td[1]/a'
            Estado_Resultados='//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-table-file-link/table/tbody/tr[2]/td[1]/a'
            Estado_Flujos_Efectivo='//*[@id="page-view"]/main/section/bvl-issuer-details/div/div/bvl-tabs/bvl-tab[5]/div/div[2]/bvl-resources/section/bvl-table-file-link/table/tbody/tr[5]/td[1]/a'
            
            #Almacenamos los estados financieros seleccionados
            Estados_Financieros={Estado_Situacion_Financiera:'SituacionFinanciera' , 
                                 Estado_Resultados:'Resultados' , 
                                 Estado_Flujos_Efectivo:'FlujoEfectivo' 
                                }

            #Finalmente obtenemos la información de cada estado
            for estados, nombre_estado in Estados_Financieros.items():
                #Seleccionar el Estado Financiero
                click_button=driver.find_element(By.XPATH, estados)
                time.sleep(3)
                click_button.click()
                time.sleep(2)

                #Expandir la tabla
                expand_button = driver.find_element(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[1]/div[2]/button/i')
                expand_button.click()

                #Obteniendo el número de filas y columnas de la tabla
                time.sleep(1)
                rows = len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr'))
                col =  len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr[1]/td'))
                time.sleep(2)
                
                
                # Crea una nueva hoja con el nombre de la empresa y el año correspondiente
                hoja_nombre = nombre_estado + str(Año_actual)
                hoja = libro.create_sheet(hoja_nombre)
                encabezado = driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr[1]/td')
                header = [elem.text for elem in encabezado]
                hoja.append(header)

                data=[]
                fila=[]

                rows = len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr'))
                col =  len(driver.find_elements(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr[1]/td'))
                time.sleep(2)
                for n in range(2, rows+1):
                    fila = []
                    for b in range(1, col+1):
                        dato = driver.find_element(By.XPATH, "/html/body/bvl-shared-modal/div/div/div/div[2]/bvl-dynamic-table/div[2]/table/tbody/tr["+str(n)+"]/td["+str(b)+"]")
                        #time.sleep(1)
                        #fila.append(dato.text)
                        #fila.append(dato.get_attribute("textContent"))
                        fila.append(dato.get_attribute("innerHTML"))
                        
                    hoja.append(fila)
                
                #Botón cerrar tabla
                close_button = driver.find_element(By.XPATH, '/html/body/bvl-shared-modal/div/div/div/div[1]/button')
                close_button.click()
                
                fila=[]
                time.sleep(2)

                
            Año_actual = Año_actual-1
     
        else:  
            Año_actual = Año_actual-1
            print()
            continue
            
        archivo_excel = nombres[i] + ".xlsx"
        libro.save(archivo_excel)
        
    # Cerrar el controlador de Selenium
    driver.quit()